In [133]:
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np
from hilbert import decode, encode
import pandas as pd
from tqdm import tqdm
import warnings
from collections import namedtuple
from functools import partial
from typing import Any, Callable, List, Optional, Tuple

import torch
import torch.nn.functional as F
from torch import nn, Tensor
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="uM0HPEvEu6OyX3dTEuB4Fihgz",
    project_name="hilbertcnn",
    workspace="romantic-brutalist",
)
import os
os.environ["COMET_API_KEY"] = "uM0HPEvEu6OyX3dTEuB4Fihgz"
hyper_params = {"batch_size": 64, "num_epochs": 5, "learning_rate": 0.0015,"trainset":"_4layer_w_volume","loss_metric":"MSE","regularizer":"L1"}
experiment.log_parameters(hyper_params)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
class HilbertImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_labels = pd.read_parquet(os.path.join(self.img_dir, annotations_file))
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path1 = os.path.join(self.img_dir, self.img_labels.iloc[idx, -1])
        img_path2 = os.path.join(self.img_dir, "s_"+self.img_labels.iloc[idx, -1])
        img_path3 = os.path.join(self.img_dir, "s8_"+self.img_labels.iloc[idx, -1])
        
        image1 = np.load(img_path1)
        image2 = np.load(img_path2)
        image3 = np.load(img_path3)
        
        
        label = self.img_labels.iloc[idx, :-1].values.astype("float64")
        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)
            image2 = self.transform(image3)
            
            
        if self.target_transform:
            label = self.target_transform(label)
        return image1,image2,image3, label


COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/romantic-brutalist/hilbertcnn/0a4b768773a14b40bc703a430ba64e98
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_batch_loss [410] : (1.1301876750167139e-05, 0.0006158257915330978)
COMET INFO:     train_loss [82]        : (1.4249053983346409e-05, 0.00012641356137141452)
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 64
COMET INFO:     learning_rate : 0.0015
COMET INFO:     loss_metric   : MSE
COMET INFO:     num_epochs    : 5
COMET INFO:     regularizer   : L1
COMET INFO:     trainset      : _4la

cuda:0


In [134]:
train_ds = HilbertImageDataset(annotations_file="target_next_20.parquet",img_dir="imgs/train"+hyper_params["trainset"]+"/")
test_ds = HilbertImageDataset(annotations_file="target_next_20.parquet",img_dir="imgs/test"+hyper_params["trainset"]+"/")
trainloader = DataLoader(train_ds, batch_size=hyper_params["batch_size"], shuffle=True)
testloader = DataLoader(test_ds, batch_size=hyper_params["batch_size"], shuffle=True)

In [135]:
a,b,c,d = next(iter(testloader))
a.shape,b.shape,c.shape,d.shape

(torch.Size([64, 5, 32, 32]),
 torch.Size([64, 5, 16, 16]),
 torch.Size([64, 5, 8, 8]),
 torch.Size([64, 8]))

In [255]:
class Net_w_16(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1a = nn.Conv2d(5, 8, 5,dilation=4,padding="same")
        torch.nn.init.xavier_uniform_(self.conv1a.weight)
        
        self.conv2a = nn.Conv2d(8, 16, 5,dilation=3,padding="same")
        torch.nn.init.xavier_uniform_(self.conv2a.weight)
        self.conv3a = nn.Conv2d(16, 24, 3,dilation=2)
        self.conv4a = nn.Conv2d(24, 48, 2)
        self.conv5a = nn.Conv2d(48, 64, 2)
        
        self.conv1b = nn.Conv2d(5, 8, 4,dilation=3,padding="same")
        torch.nn.init.xavier_uniform_(self.conv1b.weight)
        
        self.conv2b = nn.Conv2d(8, 16, 3,dilation=2,padding="same")
        torch.nn.init.xavier_uniform_(self.conv2b.weight)
        self.conv3b = nn.Conv2d(16, 24, 2,dilation=1)
        
        
        
        
        self.conv1 = nn.Conv2d(5, 8, 5,dilation=4,padding="same")
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        self.pool = nn.MaxPool2d(3, 2)
        self.pool2 = nn.AvgPool2d(3, 2)
        
        self.conv2 = nn.Conv2d(8, 16, 5,dilation=3,padding="same")
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        
        self.conv3 = nn.Conv2d(16, 24, 3,dilation=2)
        self.conv4 = nn.Conv2d(24, 48, 2)
        self.conv5 = nn.Conv2d(48, 64, 2)
        self.conv6 = nn.Conv2d(64, 128, 1)
        
        self.fc1a = nn.Linear(600, 512)
        self.fc2a = nn.Linear(512, 128)
        
        self.fc1b = nn.Linear(600, 512)
        self.fc2b = nn.Linear(512, 128)
        
        self.fc1c = nn.Linear(216, 128)
        
        
        self.fc1 = nn.Linear(1152, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        
        self.fc6 = nn.Linear(128, 64)
        self.fc7 = nn.Linear(64, 16)
        
        self.fc8 = nn.Linear(16, 8)
        
        self.batch1=nn.BatchNorm2d(8)
        self.batch2=nn.BatchNorm2d(16)
        self.batch3=nn.BatchNorm2d(24)
        self.batch4=nn.BatchNorm2d(64)
        
        self.dropout1=nn.Dropout2d(p=0.2)
        

    def forward(self, xl):
        x1=xl[0]
        x2=xl[1]
        x3=xl[2]
        
        x2 = self.dropout1(self.batch1(F.relu(self.conv1a(x2))))
        x2 = self.batch2(F.relu(self.conv2a(x2)))
        x2 = self.pool2(self.batch3(F.relu(self.conv3a(x2))))

        x3 = self.dropout1(self.batch1(F.relu(self.conv1b(x3))))
        x3 = self.batch2(F.relu(self.conv2b(x3)))
        x3 = self.pool2(self.batch3(F.relu(self.conv3b(x3))))
        
        
        
        x1 = self.dropout1(self.batch1(F.relu(self.conv1(x1))))
        x1 = self.pool(self.batch2(F.relu(self.conv2(x1))))
        x1 = self.pool2(self.batch3(F.relu(self.conv3(x1))))
        x1 = F.relu(self.conv4(x1))
        x1 = F.relu(self.conv5(x1))
        x1 = F.relu(self.conv6(x1))
        
        x1 = torch.flatten(x1, 1) # flatten all dimensions except batch
        x2 = torch.flatten(x2, 1) # flatten all dimensions except batch
        x3 = torch.flatten(x3, 1) # flatten all dimensions except batch
        #
        x1 = F.relu(self.fc1(x1))
        x1 = F.relu(self.fc2(x1))
        #x1 = F.relu(self.fc3(x1))
        
        x2 = F.relu(self.fc1a(x2))
        x2 = F.relu(self.fc2a(x2))
        
        x3 = F.relu(self.fc1c(x3))
        #print(x1.shape,x2.shape,x3.shape)
        x=torch.cat([x1,x2,x3],1)
        
        #print(x1.shape,x2.shape,x3.shape)
        
        
        x = F.leaky_relu(self.fc4(x))
        x = F.leaky_relu(self.fc5(x))
        
        x = F.relu(self.fc6(x))
        x = self.fc7(x)
        x = self.fc8(x)
        
        return x
net=Net_w_16().to(device)


In [256]:
import torch.optim as optim

criterion = torch.nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=hyper_params["learning_rate"])

In [257]:
def train(model,optimizer,criterion,dataloader,epoch):
    model.train()
    running_loss = 0.0
    total_loss= 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs1,inputs2,inputs3, labels = data
        inputs1 = inputs1.float().to(device)
        inputs2 = inputs2.float().to(device)
        inputs3 = inputs3.float().to(device)
        labels = labels.float().to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model([inputs1,inputs2,inputs3])
        loss = criterion(outputs, labels)
        

        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        total_loss += loss.item()

        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.8f}')
            experiment.log_metric("batch_loss", running_loss/200)
            running_loss = 0.0
    print("total loss:",total_loss/len(trainloader))
    experiment.log_metrics({"loss": total_loss/len(trainloader)}, epoch=epoch)

In [258]:
with experiment.train():
    print("Running Model Training")
    for epoch in range(10):
        train(net, optimizer, criterion, trainloader, epoch)

Running Model Training
[1,   200] loss: 0.00036569
[1,   400] loss: 0.00001747
[1,   600] loss: 0.00002057
[1,   800] loss: 0.00001724
[1,  1000] loss: 0.00001581
total loss: 8.009195776433589e-05
[2,   200] loss: 0.00002222
[2,   400] loss: 0.00001444
[2,   600] loss: 0.00001507
[2,   800] loss: 0.00001757
[2,  1000] loss: 0.00001466
total loss: 1.646378291134594e-05
[3,   200] loss: 0.00001632
[3,   400] loss: 0.00001351
[3,   600] loss: 0.00001607
[3,   800] loss: 0.00001490
[3,  1000] loss: 0.00001641
total loss: 1.5375164977652322e-05
[4,   200] loss: 0.00001493
[4,   400] loss: 0.00001393
[4,   600] loss: 0.00001471
[4,   800] loss: 0.00001426
[4,  1000] loss: 0.00001842
total loss: 1.5116858490153429e-05
[5,   200] loss: 0.00001438
[5,   400] loss: 0.00001310
[5,   600] loss: 0.00001635
[5,   800] loss: 0.00001421
[5,  1000] loss: 0.00001393
total loss: 1.5053633822559927e-05
[6,   200] loss: 0.00001833
[6,   400] loss: 0.00001266
[6,   600] loss: 0.00001572
[6,   800] loss: 0.0

In [261]:
from comet_ml.integration.pytorch import log_model
model_checkpoint = {
    "epoch": epoch,
    "model_state_dict": net.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "criterion":criterion}
log_model(experiment, model_checkpoint, model_name="4_layer_w_volume_w_16_8")


In [259]:
dataiter = iter(testloader)
pd.set_option('display.max_rows', 500)
cols = list(pd.concat([pd.read_parquet("imgs/train"+hyper_params["trainset"]+"/target_next_20.parquet"),
pd.read_parquet("imgs/test"+hyper_params["trainset"]+"/target_next_20.parquet")]).iloc[:,:-1].columns)
fcs = [[i+"_target",i+"_pred"] for i in cols]
fullcols = [item for sublist in fcs for item in sublist]
tl = []
for images1,images2,images3,labels in dataiter:
    images1 = images1.float().to(device)
    images2 = images2.float().to(device)
    images3 = images3.float().to(device)
    
    labels = labels.float().to(device)
    outputs = net([images1,images2,images3])
    evaldf = pd.DataFrame(labels.cpu(),columns=[i+"_target" for i in cols]).join(pd.DataFrame(outputs.cpu().tolist(),columns=[i+"_pred" for i in cols]))
    tl.append(evaldf[fullcols])
tf = pd.concat(tl)
outputs

tensor([[-3.1805e-03,  3.6973e-03, -1.7843e-03,  5.4571e-03, -1.0036e-03,
          6.2311e-03,  5.3690e-04,  6.3396e-03],
        [-3.7368e-03,  4.3886e-03, -2.2752e-03,  6.0020e-03, -1.4350e-03,
          6.7530e-03, -6.5945e-05,  6.8458e-03],
        [-1.0596e-02,  1.3559e-02, -9.7040e-03,  1.2940e-02, -7.3163e-03,
          1.2029e-02, -7.0113e-03,  1.3622e-02],
        [-2.7244e-04,  3.1722e-04,  5.7148e-04,  2.8421e-03,  1.3698e-03,
          3.5898e-03,  3.0451e-03,  3.7166e-03],
        [-4.0777e-03,  4.8020e-03, -2.6595e-03,  6.3121e-03, -1.6198e-03,
          7.0011e-03, -2.2823e-04,  7.1486e-03],
        [-1.3922e-03,  1.6295e-03, -3.5591e-04,  3.9697e-03,  2.2267e-04,
          4.5406e-03,  2.0568e-03,  4.7676e-03],
        [-4.0898e-03,  4.8471e-03, -2.6477e-03,  6.3334e-03, -1.6535e-03,
          7.0242e-03, -2.9288e-04,  7.1426e-03],
        [-3.1742e-03,  3.6226e-03, -1.7778e-03,  5.4854e-03, -9.8559e-04,
          6.1836e-03,  5.0711e-04,  6.2804e-03],
        [-2.4585

In [151]:
tf.describe()

,Low_5_target,Low_5_pred,High_5_target,High_5_pred,Low_10_target,Low_10_pred,High_10_target,High_10_pred,Low_15_target,Low_15_pred,High_15_target,High_15_pred,Low_20_target,Low_20_pred,High_20_target,High_20_pred
count,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000
mean,-0.001336,-0.003361,0.001387,0.003542,-0.001328,-0.002603,0.001413,0.003577,-0.001297,-0.004828,0.001353,0.000425,-0.001293,-0.001799,0.001344,0.002093
std,0.001608,0.005693,0.001755,0.009110,0.002745,0.009349,0.002813,0.010795,0.003449,0.008398,0.003395,0.007838,0.003944,0.004672,0.003954,0.011020
min,-0.031683,-0.045718,-0.000036,-0.107635,-0.050941,-0.116846,-0.016854,-0.051475,-0.066908,-0.088705,-0.043833,-0.066083,-0.059140,-0.079371,-0.039862,-0.055595
25%,-0.001732,-0.005694,0.000389,0.000669,-0.002212,-0.003556,0.000004,0.001555,-0.002545,-0.005371,-0.000317,-0.000225,-0.002748,-0.002045,-0.000544,-0.002685
50%,-0.000888,-0.002767,0.000913,0.001957,-0.000915,-0.001536,0.000953,0.002203,-0.000901,-0.002351,0.000942,0.001916,-0.000918,-0.001071,0.000937,0.001847
75%,-0.000365,-0.000042,0.001784,0.004489,0.000078,0.000195,0.002305,0.003640,0.000387,-0.001074,0.002582,0.002587,0.000635,-0.000442,0.002813,0.006221
max,0.000047,0.025987,0.043472,0.110648,0.025916,0.102393,0.059302,0.139139,0.036149,0.082934,0.045371,0.092297,0.036223,0.050705,0.053114,0.152165


In [260]:
tf.describe(percentiles=[i/10 for i in range(1,10)])

,Low_5_target,Low_5_pred,High_5_target,High_5_pred,Low_10_target,Low_10_pred,High_10_target,High_10_pred,Low_15_target,Low_15_pred,High_15_target,High_15_pred,Low_20_target,Low_20_pred,High_20_target,High_20_pred
count,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000
mean,-0.001336,-0.002185,0.001387,0.002481,-0.001328,-0.001126,0.001413,0.004590,-0.001297,-0.000213,0.001353,0.005238,-0.001293,0.001442,0.001344,0.005487
std,0.001608,0.003120,0.001755,0.003319,0.002745,0.003323,0.002813,0.002938,0.003449,0.002602,0.003395,0.002814,0.003944,0.002518,0.003954,0.003410
min,-0.031683,-0.036923,-0.000036,-0.001248,-0.050941,-0.045278,-0.016854,0.001550,-0.066908,-0.028638,-0.043833,0.003164,-0.059140,-0.015643,-0.039862,0.002880
10%,-0.003033,-0.005013,0.000098,-0.000031,-0.004167,-0.003481,-0.000964,0.002604,-0.004744,-0.002345,-0.001639,0.003390,-0.005337,-0.001034,-0.002157,0.003445
20%,-0.002023,-0.003232,0.000293,0.000325,-0.002694,-0.001826,-0.000228,0.002839,-0.003058,-0.001022,-0.000656,0.003596,-0.003362,0.000491,-0.000957,0.003720
30%,-0.001516,-0.002411,0.000485,0.000650,-0.001865,-0.001147,0.000216,0.003045,-0.002109,-0.000442,-0.000023,0.003782,-0.002289,0.001248,-0.000209,0.003957
40%,-0.001155,-0.001710,0.000692,0.000954,-0.001361,-0.000617,0.000601,0.003250,-0.001449,0.000011,0.000466,0.003962,-0.001554,0.001830,0.000384,0.004173
50%,-0.000888,-0.001184,0.000913,0.001381,-0.000915,-0.000081,0.000953,0.003661,-0.000901,0.000476,0.000942,0.004313,-0.000918,0.002321,0.000937,0.004504
60%,-0.000649,-0.000770,0.001194,0.001926,-0.000519,0.000220,0.001386,0.004234,-0.000409,0.000861,0.001500,0.004818,-0.000361,0.002681,0.001586,0.005025


In [248]:
tf.describe(percentiles=[i/10 for i in range(1,10)])

,Low_5_target,Low_5_pred,High_5_target,High_5_pred,Low_10_target,Low_10_pred,High_10_target,High_10_pred,Low_15_target,Low_15_pred,High_15_target,High_15_pred,Low_20_target,Low_20_pred,High_20_target,High_20_pred
count,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000,7916.000000
mean,-0.001336,-0.002470,0.001387,0.002068,-0.001328,-0.003138,0.001413,0.000726,-0.001297,-0.001292,0.001353,0.001696,-0.001293,-0.003519,0.001344,-0.000568
std,0.001608,0.000970,0.001755,0.000965,0.002745,0.000699,0.002813,0.000953,0.003449,0.000787,0.003395,0.000790,0.003944,0.000719,0.003954,0.000972
min,-0.031683,-0.008992,-0.000036,0.000716,-0.050941,-0.005699,-0.016854,-0.000715,-0.066908,-0.010059,-0.043833,0.000114,-0.059140,-0.007830,-0.039862,-0.001956
10%,-0.003033,-0.003730,0.000098,0.001132,-0.004167,-0.004210,-0.000964,-0.000228,-0.004744,-0.002238,-0.001639,0.000833,-0.005337,-0.004558,-0.002157,-0.001543
20%,-0.002023,-0.003119,0.000293,0.001369,-0.002694,-0.003774,-0.000228,-0.000012,-0.003058,-0.001844,-0.000656,0.001065,-0.003362,-0.004170,-0.000957,-0.001294
30%,-0.001516,-0.002715,0.000485,0.001519,-0.001865,-0.003447,0.000216,0.000140,-0.002109,-0.001558,-0.000023,0.001217,-0.002289,-0.003846,-0.000209,-0.001133
40%,-0.001155,-0.002419,0.000692,0.001640,-0.001361,-0.003183,0.000601,0.000280,-0.001449,-0.001298,0.000466,0.001340,-0.001554,-0.003561,0.000384,-0.001002
50%,-0.000888,-0.002183,0.000913,0.001809,-0.000915,-0.002969,0.000953,0.000468,-0.000901,-0.001092,0.000942,0.001508,-0.000918,-0.003325,0.000937,-0.000818
60%,-0.000649,-0.002011,0.001194,0.002041,-0.000519,-0.002805,0.001386,0.000709,-0.000409,-0.000954,0.001500,0.001718,-0.000361,-0.003162,0.001586,-0.000565


In [254]:
tf[tf.Low_5_pred*0.5<tf.Low_5_target]

,Low_5_target,Low_5_pred,High_5_target,High_5_pred,Low_10_target,Low_10_pred,High_10_target,High_10_pred,Low_15_target,Low_15_pred,High_15_target,High_15_pred,Low_20_target,Low_20_pred,High_20_target,High_20_pred
0,-0.000453,-0.002150,0.002245,0.001788,0.002017,-0.003044,0.004239,0.000423,0.002554,-0.001168,0.004029,0.001572,0.002792,-0.003460,0.004602,-0.000761
1,-0.001320,-0.002920,0.005935,0.002476,-0.000186,-0.003489,0.004533,0.001044,0.000867,-0.001636,0.003199,0.002041,-0.001139,-0.003969,0.003226,-0.000177
2,0.000000,-0.001993,0.004065,0.001630,0.001436,-0.002768,0.005135,0.000183,0.003342,-0.000987,0.004743,0.001299,0.003659,-0.003179,0.008794,-0.001045
3,-0.000169,-0.002720,0.001787,0.002332,0.000807,-0.003573,0.003072,0.001143,0.001618,-0.001680,0.003105,0.002096,0.001735,-0.004061,0.002641,-0.000102
4,-0.000004,-0.001524,0.000684,0.001169,-0.000096,-0.002354,0.000595,-0.000280,-0.000910,-0.000383,0.001967,0.000880,0.001245,-0.002675,0.003342,-0.001586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,-0.000878,-0.003497,0.000831,0.003066,0.000549,-0.004264,0.007772,0.001963,0.004140,-0.002355,0.006094,0.002758,0.005321,-0.004550,0.008567,0.000463
32,-0.000032,-0.002300,0.005154,0.001942,0.004327,-0.003206,0.006003,0.000673,0.004191,-0.001240,0.010126,0.001711,0.007519,-0.003585,0.010094,-0.000583
36,0.000020,-0.002132,0.002205,0.001757,0.000609,-0.002948,0.002858,0.000365,0.000609,-0.001173,0.002554,0.001479,0.000095,-0.003395,0.001836,-0.000839
38,-0.000981,-0.003234,0.001764,0.002696,-0.001173,-0.003933,-0.000056,0.001398,-0.001906,-0.001752,0.000399,0.002516,-0.000738,-0.004377,0.001259,0.000083


In [251]:
tf

,Low_5_target,Low_5_pred,High_5_target,High_5_pred,Low_10_target,Low_10_pred,High_10_target,High_10_pred,Low_15_target,Low_15_pred,High_15_target,High_15_pred,Low_20_target,Low_20_pred,High_20_target,High_20_pred
0,-0.000453,-0.002150,0.002245,0.001788,0.002017,-0.003044,0.004239,0.000423,0.002554,-0.001168,0.004029,0.001572,0.002792,-0.003460,0.004602,-0.000761
1,-0.001320,-0.002920,0.005935,0.002476,-0.000186,-0.003489,0.004533,0.001044,0.000867,-0.001636,0.003199,0.002041,-0.001139,-0.003969,0.003226,-0.000177
2,0.000000,-0.001993,0.004065,0.001630,0.001436,-0.002768,0.005135,0.000183,0.003342,-0.000987,0.004743,0.001299,0.003659,-0.003179,0.008794,-0.001045
3,-0.000169,-0.002720,0.001787,0.002332,0.000807,-0.003573,0.003072,0.001143,0.001618,-0.001680,0.003105,0.002096,0.001735,-0.004061,0.002641,-0.000102
4,-0.000004,-0.001524,0.000684,0.001169,-0.000096,-0.002354,0.000595,-0.000280,-0.000910,-0.000383,0.001967,0.000880,0.001245,-0.002675,0.003342,-0.001586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,-0.003360,-0.001644,0.000437,0.001223,-0.002313,-0.002449,-0.001531,-0.000050,-0.002679,-0.000479,-0.001813,0.000997,-0.003096,-0.002816,-0.001977,-0.001444
40,-0.002022,-0.001400,0.000306,0.000961,-0.002284,-0.002230,-0.001659,-0.000386,-0.001882,-0.000273,-0.000755,0.000696,-0.003187,-0.002548,-0.001205,-0.001721
41,-0.003124,-0.002132,0.001012,0.001767,-0.002170,-0.002962,0.000145,0.000445,-0.000731,-0.001075,0.001681,0.001519,0.000000,-0.003337,0.003299,-0.000826
42,-0.003117,-0.003202,0.002039,0.002754,-0.007422,-0.003990,-0.000515,0.001676,-0.007224,-0.001865,-0.001667,0.002575,-0.008085,-0.004315,-0.002721,0.000245
